In [1]:
#lol y'a pas de module pour le torus on est dans la merde

In [2]:
# si k E Tp , il existe un entier r E [0;p] tq k = r/p
# on peut donc considérer k comme (r,p) ou juste comme r jsp

In [3]:
def AddInTore(p,a,b) :
    return (a+b)%p

In [4]:
# x in TP
# n in Z
def MultExtTore(p,n,x):
    if(n>=0):
        cpt = 0
        for i in range(n):
            cpt = AddInTore(p,cpt,x)
        return cpt
    return MultExtTore(p,-n,-x)

In [5]:
def TLWEncrypt(p,q,n,s,sigma,m):
    a=[]
    for i in range(n):
        a.append(randint(0,q-1))
    e = RealDistribution("gaussian",sigma).get_random_element()
    e = round(q*e)
    while(abs(e/q)>=1/(2*p)):
        e = RealDistribution("gaussian",sigma).get_random_element()
        e = round(q*e)
    #print("e= ",e)
    c = AddInTore(q,m*q/p,e)
    for i in range(n):
        c = AddInTore(q,c,MultExtTore(q,s[i],a[i]))
    return a+[c]

In [6]:
def TLWEDecrypt(p,q,n,s,c):
    a = c[:-1]
    m = c[-1]
    for i in range(n):
        m = AddInTore(q,m,MultExtTore(q,-s[i],a[i]))
    return round(m*p/q)%p

In [7]:
p = 16
q = 64
n = 5
s = [0,1,1,0,0]
sigma = 0.001

m = 10

c = TLWEncrypt(p, q, n, s, sigma, m)
print("c =",c)

mmaybe = TLWEDecrypt(p, q, n, s, c)
print("m =",mmaybe)

c = [1, 7, 17, 28, 33, 0]
m = 10


In [9]:
p = 2
q = 2**32
n = 1000
s = [randint(0,1) for i in range(n)]
sigma = 2**(-7)

cpt = 0
for i in range(1000):
    
    m = randint(0,p-1)


    c = TLWEncrypt(p, q, n, s, sigma, m)

    mmaybe = TLWEDecrypt(p, q, n, s, c)
    if(m==mmaybe):
        cpt = cpt +1
print(cpt)        

1000


In [10]:
def AddCiph(q,n,c1,c2):
    return [AddInTore(q,c1[i],c2[i]) for i in range(n+1)]

In [11]:
p = 2**9
q = 2**16
n = 1000
s = [randint(0,1) for i in range(n)]
sigma = 2**(-7)

cpt1 = 0
cpt2 = 0
for i in range(1000):
    
    m1 = randint(0,p-1)
    m2 = randint(0,p-1)

    c1,e1 = TLWEncrypt(p, q, n, s, sigma, m1)
    c2,e2 = TLWEncrypt(p, q, n, s, sigma, m2)

    m = TLWEDecrypt(p, q, n, s, AddCiph(q,n,c1,c2))
    if(m==AddInTore(p,m1,m2)):
        cpt1 = cpt1 +1
    elif(abs((e1+e2)/q)>=1/(2*p)):
        cpt2 = cpt2 + 1
print(cpt1)
print(cpt2)
print(cpt1+cpt2 ==1000)

ValueError: too many values to unpack (expected 2)

In [12]:
def MultCiph(q,n,K,c):
    return [MultExtTore(q,K,c[i]) for i in range(n+1)]

In [13]:
p = 2
q = 2**32
n = 1000
s = [randint(0,1) for i in range(n)]
sigma = 2**(-7)

cpt1 = 0
cpt2 = 0
for i in range(1000):
    
    m = randint(0,p-1)

    c,e = TLWEncrypt(p, q, n, s, sigma, m)
    
    K = randint(1,100)
    mmaybe = TLWEDecrypt(p, q, n, s, MultCiph(q,n,K,c))
    
    if(mmaybe==MultExtTore(p,K,m)):
        cpt1 +=1
    elif(abs((K*e)/q)>=1/(2*p)):
        cpt2 +=1
print(cpt1)
print(cpt2)
print(cpt1+cpt2 == 1000)

ValueError: too many values to unpack (expected 2)

In [14]:
def TrueGadgetMatrix(q,B,l,n):
    G = [[0 for i in range(n+1)] for j in range((n+1)*l)]
    for i in range(n+1):
        for j in range(0,l):
            G[j+l*i][i] = 1/(B^(j+1))
    return G
#G is the real gadget matrix

In [15]:
def UsedGadgetMatrix(q,B,l,n):
    G = [[0 for i in range(n+1)] for j in range((n+1)*l)]
    for i in range(n+1):
        for j in range(0,l):
            G[j+l*i][i] = q/(B^(j+1))
    return G
#G is the gadhet matrix of the numerator of the real gadget matrix with q for denominator

In [16]:
def AddInMatr(q,M1,M2):
    return [[AddInTore(q,M1[i][j],M2[i][j]) for j in range(len(M1[0]))] for i in range(len(M1))]

In [35]:
#Works with M a matrix with coeff in our Tq
def MultExtMatr(q,K,M):
    return [[MultExtTore(q,K,M[i][j]) for j in range(len(M[0]))] for i in range(len(M))] 

In [18]:
#M1 matrice entière et M2 matrice dans Tq
def MultInMatr(q,M1,M2) :
    M = [[0 for _ in range(len(M2[0]))] for _ in range(len(M1))]
    for i in range(len(M1)) :
        for j in range(len(M2[0])) :
            for k in range(len(M2)) :
                M[i][j] = AddInTore(q,M[i][j],MultExtTore(q,M1[i][k],M2[k][j]))
    return M

In [19]:
def TGSWEncrypt(q,B,l,n,m,s):
    sigma = 0.001
    Z = [TLWEncrypt(B**l,q,n,s,sigma,0) for i in range(n+1)]
    #print(Z)
    return AddInMatr(q,Z,MultExtMatr(q,m,UsedGadgetMatrix(q,B,l,n)))

In [20]:
def InvGVect(q,B,l,n,v):
    for i in range(n+1) :
        if v[i]> q/2 :
            v[i] = v[i]-q
    vbar = [round(B**l*v[i]/q) for i in range(n+1)]
    u=[]
    for i in range(n+1) :
        tmp = (vbar[i]).digits(B)
        while (len(tmp)!=l+1) :
            tmp.append(0)
        for j in range(len(tmp)-1) :
            if tmp[j]>= ceil(B/2) :
                tmp[j] = tmp[j]-B
                tmp[j+1] = tmp[j+1]+1
        tmp.reverse()
        u = u+(tmp[1:])
    return [u]

In [21]:
def InvGMatr(q,B,l,n,M) :
    return [InvGVect(q,B,l,n,M[i]) for i in range(len(M))]

In [24]:
g = InvGVect(64,4,3,1,[41,26])
G = UsedGadgetMatrix(64,4,3,1)
MultInMatr(64,g,G)

[[41, 26]]

In [25]:
def MultInGSW(q,B,l,n,c1,c2) :
    return MultInMatr(q,InvGMatr(q,B,l,n,c1),c2)

In [26]:
#c1 est TGSW
#c2 est TLWE
def MultExtCiphLWE(q,B,l,n,c1,c2) :
    return MultInMatr(q,InvGVect(q,B,l,n,c2),c1)

In [27]:
k = 3
m = 5
q = 64
B = 4
l = 2
p = B**l
n = 500
sigma = 2**(-10)
cpt=0
for j in range(1000) :
    s = [randint(0,1) for i in range(n)]
    cm = TLWEncrypt(p,q,n,s,sigma,m)

    ck = TGSWEncrypt(q,B,l,n,m,s)
    c = MultExtCiphLWE(q,B,l,n,ck,cm)[0]
    if(TLWEDecrypt(p,q,n,s,c)==MultExtTore(p,k,m)) :
        cpt+=1
print(cpt)


KeyboardInterrupt: 

## TGLWE

In [28]:
# p1,p2 defined in Tp[X] / (X^N+1)
def AddPolyInTore(q,p1,p2,N) :
    p = [0]*N
    for i in range(N) :
        p[i] = AddInTore(q,p1[i],p2[i])
    return p

In [29]:
# p1 in TP[X] / X^N+1
# p2 in Z[X] / X^N+1
def MultPolyExt(q,p1,p2,N) :
    P = [0]*2*N
    for i in range(N) :
        for j in range(N) :
            tmp = MultExtTore(q,p2[i],p1[j])
            P[i+j] = AddInTore(q,P[i+j],tmp)
    for i in range(N) :
        P[i] = AddInTore(q,P[i],-P[i+N])
    return P[:N]

In [45]:
def TGLWEncrypt(p,q,N,k,s,sigma,m) :
    a = []
    for i in range(k) :
        a.append([randint(0,q-1) for j in range(N)])
    e = [RealDistribution("gaussian",sigma).get_random_element() for i in range(N)]
    e = [round(q*e[i]) for i in range(N)]
    M = [m[i]*q/p for i in range(N)]
    c = AddPolyInTore(q,M,e,N)
    for i in range(k):
        c = AddPolyInTore(q,c,MultPolyExt(q,s[i],a[i],N),N)
    return a+[c]

In [46]:
TGLWEncrypt(4,16,4,3,[[0,1,1,1],[1,0,0,1],[0,0,0,1]],2**-7,[1,3,3,2])

[[0, 5, 15, 15], [9, 10, 14, 6], [7, 5, 5, 13], [11, 5, 13, 2]]

In [32]:
N = 4
q = 8
P1 = [1,0,0,2]
P2 = [3,5,0,2]
MultPolyExt(q,P1,P2,N)

[1, 5, 4, 0]

In [62]:
def AddInMatrPoly(q,M1,M2,N):
    return [[AddPolyInTore(q,M1[i][j],M2[i][j],N) for j in range(len(M1[0]))] for i in range(len(M1))]

In [80]:
#Works with M a matrix with coeff in our Tq
#and K in Z/pZ[X]

#Return K x M 
def MultExtMatrPoly(q,K,M,N):
    return [[[MultExtTore(q,K[k],M[i][j]) for k in range(N)] for j in range(len(M[0]))] for i in range(len(M))] 

In [87]:
def TGGSWEncrypt(q,B,l,N,k,s,m):
    sigma = 0.001
    Z = [TGLWEncrypt(B**l,q,N,k,s,sigma,[0 for _ in range(N)]) for i in range(k+1)]
    #print(Z)
    return AddInMatrPoly(q,Z,MultExtMatrPoly(q,m,UsedGadgetMatrix(q,B,l,k),N),N)

In [89]:
q =64
B = 4
l = 2
k = 2
N = 2
m = [1,3]
sigma = 0.001
s = [[0,1],[1,0],[0,0]]
#print(UsedGadgetMatrix(q,B,l,k))
#print(MultExtMatrPoly(q,m,UsedGadgetMatrix(q,B,l,k),N))
Z = [TGLWEncrypt(B**l,q,N,k,s,sigma,[0 for _ in range(N)]) for i in range(k+1)]
print(Z)

[[[8, 48], [0, 63], [16, 7]], [[50, 28], [10, 59], [46, 45]], [[19, 33], [31, 48], [62, 3]]]


In [90]:
TGGSWEncrypt(16,2,2,2,3,[[0,1],[1,0],[0,0]],[1,3])

[[[13, 1], [6, 12], [2, 9], [13, 1]],
 [[4, 9], [2, 14], [14, 6], [5, 14]],
 [[9, 1], [9, 13], [8, 6], [0, 14]],
 [[6, 12], [8, 6], [11, 2], [8, 0]]]

In [43]:
G = UsedGadgetMatrix(64,4,3,2)
K = [1,2,3,4]
MultExtMatrPoly(64,K,G,4)

[[[16, 32, 48, 0], [0, 0, 0, 0], [0, 0, 0, 0]],
 [[4, 8, 12, 16], [0, 0, 0, 0], [0, 0, 0, 0]],
 [[1, 2, 3, 4], [0, 0, 0, 0], [0, 0, 0, 0]],
 [[0, 0, 0, 0], [16, 32, 48, 0], [0, 0, 0, 0]],
 [[0, 0, 0, 0], [4, 8, 12, 16], [0, 0, 0, 0]],
 [[0, 0, 0, 0], [1, 2, 3, 4], [0, 0, 0, 0]],
 [[0, 0, 0, 0], [0, 0, 0, 0], [16, 32, 48, 0]],
 [[0, 0, 0, 0], [0, 0, 0, 0], [4, 8, 12, 16]],
 [[0, 0, 0, 0], [0, 0, 0, 0], [1, 2, 3, 4]]]

In [42]:
G

[[16, 0, 0],
 [4, 0, 0],
 [1, 0, 0],
 [0, 16, 0],
 [0, 4, 0],
 [0, 1, 0],
 [0, 0, 16],
 [0, 0, 4],
 [0, 0, 1]]